# 编码器-解码器
编码器：将文本表示成向量
解码器：将向量表示成输出

## 编码器-解码器架构
一个模型被分为两块：
    1. 编码器处理输入，将Input变成中间状态的state
    2. 解码器生成输出
<img alt="image.png" height="600" src="/Users/loneker/Desktop/d2l/pic/iShot_2024-05-15_15.05.10.png" width="700"/>

## 总结
使用编码器-解码器架构的模型，编码器负责表示输入，解码器负责输出

In [1]:
from torch import nn

class Encoder(nn.Module):
    """编码器-解码器架构的基本编码器接口"""
    def __init__(self, **kwargs):
        super(Encoder, self).__init__(**kwargs)

    def forward(self, X, *args):        # 给定X输出结果
        raise NotImplementedError

## [**解码器**]

在下面的解码器接口中，我们新增一个`init_state`函数，
用于将编码器的输出（`enc_outputs`）转换为编码后的状态。
注意，此步骤可能需要额外的输入，例如：输入序列的有效长度，
这在 :numref:`subsec_mt_data_loading`中进行了解释。
为了逐个地生成长度可变的词元序列，
解码器在每个时间步都会将输入
（例如：在前一时间步生成的词元）和编码后的状态
映射成当前时间步的输出词元。


In [2]:
#@save
class Decoder(nn.Module):
    """编码器-解码器架构的基本解码器接口"""
    def __init__(self, **kwargs):
        super(Decoder, self).__init__(**kwargs)

    def init_state(self, enc_outputs, *args):       # 编码器的输出，不同的模型代码会不一样
        raise NotImplementedError   

    def forward(self, X, state):                    # X：拿到额外的输入， state：从encoder那里拿到，并且可以一直更新
        raise NotImplementedError

In [4]:
# 合并编码器和解码器
class EncoderDecoder(nn.Module):
    """编码器-解码器架构的基类"""
    def __init__(self, encoder, decoder, **kwargs):
        super(EncoderDecoder, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, enc_X, dec_X, *args):
        enc_outputs = self.encoder(enc_X, *args)
        dec_state = self.decoder.init_state(enc_outputs, *args)     # 拿到编码器的输出
        return self.decoder(dec_X, dec_state)                       # 将编码器的输出和解码器的输入一起当作输入